# 函数式编程
函数是Python内建支持的一种封装，我们通过把大段代码拆成函数，通过一层一层的函数调用，就可以把复杂任务分解成简单的任务，这种分解可以称之为面向过程的程序设计。函数就是面向过程的程序设计的基本单元。函数式编程就是一种抽象程度很高的编程范式，纯粹的函数式编程语言编写的函数没有变量，因此，任意一个函数，只要输入是确定的，输出就是确定的，这种纯函数我们称之为没有副作用。而允许使用变量的程序设计语言，由于函数内部的变量状态不确定，同样的输入，可能得到不同的输出，因此，这种函数是有副作用的。

函数式编程的一个特点就是，允许把函数本身作为参数传入另一个函数，还允许返回一个函数！

Python对函数式编程提供部分支持。由于Python允许使用变量，因此，Python不是纯函数式编程语言。

## 高阶函数
### 变量可以指向函数

In [11]:
abs

<function abs(x, /)>

可见，abs(-10)是函数调用，而abs是函数本身。

要获得函数调用结果，我们可以把结果赋值给变量

但是，如果把函数本身赋值给变量呢？

In [12]:
f=abs
f

<function abs(x, /)>

结论：函数本身也可以赋值给变量，即：变量可以指向函数。

成功！说明变量f现在已经指向了abs函数本身。直接调用abs()函数和调用变量f()完全相同。

## 函数名也是变量
那么函数名是什么呢？函数名其实就是指向函数的变量！对于abs()这个函数，完全可以把函数名abs看成变量，它指向一个可以计算绝对值的函数！

In [ ]:
abs=10
abs(-10) #注意执行后环境内abs函数将被改变, 需要重新启动环境来重新将abs绑定回abs函数

把abs指向10后，就无法通过abs(-10)调用该函数了！因为abs这个变量已经不指向求绝对值函数而是指向一个整数10！

当然实际代码绝对不能这么写，这里是为了说明函数名也是变量。要恢复abs函数，请重启Python交互环境。

注：由于abs函数实际上是定义在import builtins模块中的，所以要让修改abs变量的指向在其它模块也生效，要用import builtins; builtins.abs = 10。

## 传入函数
既然变量可以指向函数，函数的参数能接收变量，那么一个函数就可以接收另一个函数作为参数，这种函数就称之为高阶函数。

一个最简单的高阶函数：当我们调用add(-5, 6, abs)时，参数x，y和f分别接收-5，6和abs

In [16]:
def add(x, y, f):
    return f(x) + f(y)
add(-5,6,abs)

11

## map和reduce函数
# map函数
map()函数接收两个参数，一个是函数，一个是Iterable，map将传入的函数依次作用到序列的每个元素，并把结果作为新的Iterator返回。

举例说明，比如我们有一个函数f(x)=x2，要把这个函数作用在一个list [1, 2, 3, 4, 5, 6, 7, 8, 9]上

In [19]:
def squre(x):
    return x*x
list(map(squre,list(range(1,10))))

[1, 4, 9, 16, 25, 36, 49, 64, 81]

map()传入的第一个参数是f，即函数对象本身。由于结果r是一个Iterator，Iterator是惰性序列，因此通过list()函数让它把整个序列都计算出来并返回一个list。

# reduce函数
reduce把一个函数作用在一个序列[x1, x2, x3, ...]上，这个函数必须接收两个参数，reduce把结果继续和序列的下一个元素做累积计算，其效果就是：

reduce(f, [x1, x2, x3, x4]) = f(f(f(x1, x2), x3), x4)

In [30]:
from functools import reduce
def add(x,y):  # 累加函数
    return x+y
print(reduce(add,list(range(1,10))))

def func1(x,y):  # 将所有数字合并成一个数字
    return 10*x+y
print(reduce(func1,list(range(1,10))))

digits={'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9}
def str2num(s):
    return digits[s]
def str2int(s):
    return reduce(lambda x,y:10*x+y, map(str2num,s))
print(str2int('123456789'))

45
123456789
123456789


# filter函数
filter()也接收一个函数和一个序列。和map()不同的是，filter()把传入的函数依次作用于每个元素，然后根据返回值是True还是False决定保留还是丢弃该元素。

In [39]:
def is_odd(s):  # 创建一个筛选出奇数的函数
    return s%2==1
print(list(filter(is_odd,list(range(1,10)))))

def not_empty(s):
    return s and s.strip()
print(list(filter(not_empty,['1','2','3','4',' ','r',None])))

[1, 3, 5, 7, 9]
['1', '2', '3', '4', 'r']


注意到filter()函数返回的是一个Iterator，也就是一个惰性序列，所以要强迫filter()完成计算结果，需要用list()函数获得所有结果并返回list。

In [3]:
#用filter求素数(埃氏筛法)
def odd_num():
    n=1
    while True:
        n=n+2
        yield n

def not_prime(s):
    return lambda x:x%s>0

def primes():
    yield 2
    odd=odd_num()
    while True:
        a=next(odd)
        yield a
        odd=filter(not_prime(a),odd)
        
for i in primes():
    if i<50:
        print(i)
    else:
        break

2
3
5
7
11
13
17
19
23
29
31
37
41
43
47


# sort 排序算法
排序也是在程序中经常用到的算法。无论使用冒泡排序还是快速排序，排序的核心是比较两个元素的大小。如果是数字，我们可以直接比较，但如果是字符串或者两个dict呢？直接比较数学上的大小是没有意义的，因此，比较的过程必须通过函数抽象出来。

Python内置的sorted()函数就可以对list进行排序：

In [5]:
print(sorted([36, 5, -12, 9, -21]))

[-21, -12, 5, 9, 36]


此外，sorted()函数也是一个高阶函数，它还可以接收一个key函数来实现自定义的排序，key指定的函数将作用于list的每一个元素上，并根据key函数返回的结果进行排序。然后sorted()函数按照keys进行排序，例如按绝对值大小排序.

要进行反向排序, 不必改动key函数，可以传入第三个参数reverse=True

In [20]:
print(sorted([36,5,-12,9,-21],key=abs))
print(sorted([36,5,-12,9,-21],key=abs,reverse=True))

[5, 9, -12, -21, 36]
[36, -21, -12, 9, 5]


默认情况下sorted对字符串的排序是按照ASCII的大小比较的，由于'Z' < 'a'，结果，大写字母Z会排在小写字母a的前面。

In [14]:
sorted(['bob', 'about', 'Zoo', 'Credit'])

['Credit', 'Zoo', 'about', 'bob']

现在，我们提出排序应该忽略大小写，按照字母序排序。要实现这个算法，不必对现有代码大加改动，只要我们能用一个key函数把字符串映射为忽略大小写排序即可。忽略大小写来比较两个字符串，实际上就是先把字符串都变成大写（或者都变成小写），再比较。

这样，我们给sorted传入key函数，即可实现忽略大小写的排序：

In [15]:
sorted(['bob', 'about', 'Zoo', 'Credit'], key=str.lower)

['about', 'bob', 'Credit', 'Zoo']

# 返回函数
## 函数作为返回值
高阶函数除了可以接受函数作为参数外，还可以把函数作为结果值返回。

我们来实现一个可变参数的求和。

In [2]:
def calc_sum(*args): 
    ax = 0
    for n in args:
        ax = ax + n
    return ax  # 返回求和的结果

def return_sum(*args):
    def sum():
        ax = 0
        for n in args:
            ax = ax + n
        return ax
    return sum
g=calc_sum(1,2,3,4,5,6,7,8,9)
f=return_sum(1,2,3,4,5,6,7,8,9)
print(g)
print(f)
print(f()) # 调用函数f()时, 才真正计算求和结果

45
<function return_sum.<locals>.sum at 0x000002A8344DF1E0>
45


在这个例子中，我们在函数lazy_sum中又定义了函数sum，并且，内部函数sum可以引用外部函数lazy_sum的参数和局部变量，当lazy_sum返回函数sum时，相关参数和变量都保存在返回的函数中，这种称为“闭包（Closure）”的程序结构拥有极大的威力。

# 闭包


闭包的作用

一般情况下,函数中的局部变量仅在函数的执行期间可用, 一旦外函数执行过后, 我们认为局部变量将不再可用, 但是闭包使得局部变量在函数外的访问成为可能.(避免了使用全局变量,并且允许我们将某些数据与一个或者多个方法相关联

注意到返回的函数在其定义内部引用了局部变量args，所以，当一个函数返回了一个函数后，其内部的局部变量还被新函数引用，所以，闭包用起来简单，实现起来可不容易。

另一个需要注意的问题是，返回的函数并没有立刻执行，而是直到调用了f()才执行。我们来看一个例子：

In [24]:
def count():
    fs=[]
    for i in range(1,4):
        def f():
            return i*i
        fs.append(f)
    return fs
f1,f2,f3=count()
print(f1(),f2(),f3())

9 9 9


全部都是9！原因就在于返回的函数引用了变量i，但它并非立刻执行。等到3个函数都返回时，它们所引用的变量i已经变成了3，因此最终结果为9。

### 引用循环变量

方法是再创建一个函数，用该函数的参数绑定循环变量当前的值，无论该循环变量后续如何更改，已绑定到函数参数的值不变：

In [22]:
def count():
    def f(j):
        def g():
            return j*j
        return g
    fs=[]
    for i in range(1,4):
        fs.append(f(i))
    return fs
f1,f2,f3=count()
print(f1(),f2(),f3())

1 4 9
